In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/bigdata/

/content/drive/MyDrive/bigdata


In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 79kB/s 
     |████████████████████████████████| 204kB 51.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=b1bd397b54b23f097c98f5be85f00c40787469e7b9f5c80d5a295f61b56f78eb
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, CountVectorizer
from pyspark.sql.functions import col, udf
import pandas as pd

# Save csv file to parquet

### Preprocessing articles

In [ ]:
df = pd.read_csv('./data/articles.csv')
df.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,title,sapo,url,source,pega_cate_id,title_token,sapo_token,content_token,all_token
0,6.366651e+17,Anh phát hiện 39 thi thể trong một xe tải,"Ngày 23/10, cảnh sát Anh đã phát hiện 39 thi t...",http://vnmedia.vn/quoc-te/201910/anh-phat-hien...,vnmedia.vn,102,Anh phát_hiện 39 thi_thể trong một xe_tải,"Ngày 23/10 , cảnh_sát Anh đã phát_hiện 39 thi_...","Theo cảnh_sát địa_phương , chiếc xe_tải trên đ...",anh phát_hiện thi_thể trong một xe_tải ngày...
1,6.368641e+17,Phát hiện két sắt chứa 18 kg vàng dưới con đập vỡ,"Theo TASS ngày 23-10 đưa tin, trong quá trình ...",http://congan.com.vn/quoc-te/chuyen-bon-phuong...,congan.com.vn,102,Phát_hiện két sắt chứa 18 kg vàng dưới con đập vỡ,"( CAO ) Theo TASS ngày 23-10 đưa tin , trong q...","Theo điều_tra ban_đầu , chiếc két sắt này nằm ...",phát_hiện két sắt chứa kg vàng dưới con đập...
2,6.370352e+17,"Máy bay rơi ở Mexico, ít nhất 5 người thiệt mạng","Theo Sputnik ngày 24-10 đưa tin, chiếc máy bay...",http://congan.com.vn/quoc-te/thoi-su-quoc-te/m...,congan.com.vn,102,"Máy_bay rơi ở Mexico , ít_nhất 5 người thiệt_mạng","( CAO ) Theo Sputnik ngày 24-10 đưa tin , chiế...",Chiếc máy_bay đang trong hành_trình từ bang Du...,máy_bay rơi ở mexico ít_nhất người thiệt_m...
3,6.372364e+17,Hình ảnh Đệ nhất phu nhân Triều Tiên xinh đẹp ...,- Giữa lúc tin đồn đang rộ lên xung quanh việc...,http://vnmedia.vn/tin-anh/201910/hinh-anh-de-n...,vnmedia.vn,102,Hình_ảnh Đệ nhất phu_nhân Triều_Tiên xinh_đẹp ...,- Giữa lúc tin_đồn đang rộ lên xung_quanh việc...,HÌnh_ảnh Đệ nhất phu_nhân xinh_đẹp của Triều_T...,hình_ảnh đệ nhất phu_nhân triều_tiên xinh_đẹp ...
4,6.380423e+17,Thủ lĩnh cao nhất của IS nghi bị tiêu diệt tro...,"​(CAO) Hôm 27-10, Reuters đưa tin thủ lĩnh tối...",http://congan.com.vn/quoc-te/thoi-su-quoc-te/t...,congan.com.vn,102,Thủ_lĩnh cao nhất của IS nghi bị tiêu_diệt tro...,"​ ( CAO ) Hôm 27-10 , Reuters đưa tin thủ_lĩnh...","Các nguồn_tin ở Syria , Iraq và Iran đã tiết_l...",thủ_lĩnh cao nhất của is nghi bị tiêu_diệt tro...


In [ ]:
df_label = pd.read_csv('./data/cate_info.csv')

In [ ]:
df_label.head()

,id,name,parent_id
0,101,Xã hội,1
1,102,Thế giới,1
2,103,Tình yêu,2
3,104,Nóng trên mạng,2
4,105,Âm nhạc,3


In [ ]:
from tqdm import tqdm

cate_dict = dict()
name = df_label['name'].values
id = df_label['id'].values
for i in range(len(id)):
    cate_dict[id[i]] = name[i]
cate_id = df['pega_cate_id'].values
cate_name = []
for i in tqdm(range(len(cate_id))):
    cate_name.append(cate_dict[cate_id[i]])
df['label'] = cate_name
df

100%|██████████| 84230/84230 [00:00<00:00, 1239844.41it/s]


,id,title,sapo,url,source,pega_cate_id,title_token,sapo_token,content_token,all_token,label
0,6.366651e+17,Anh phát hiện 39 thi thể trong một xe tải,"Ngày 23/10, cảnh sát Anh đã phát hiện 39 thi t...",http://vnmedia.vn/quoc-te/201910/anh-phat-hien...,vnmedia.vn,102,Anh phát_hiện 39 thi_thể trong một xe_tải,"Ngày 23/10 , cảnh_sát Anh đã phát_hiện 39 thi_...","Theo cảnh_sát địa_phương , chiếc xe_tải trên đ...",anh phát_hiện thi_thể trong một xe_tải ngày...,Thế giới
1,6.368641e+17,Phát hiện két sắt chứa 18 kg vàng dưới con đập vỡ,"Theo TASS ngày 23-10 đưa tin, trong quá trình ...",http://congan.com.vn/quoc-te/chuyen-bon-phuong...,congan.com.vn,102,Phát_hiện két sắt chứa 18 kg vàng dưới con đập vỡ,"( CAO ) Theo TASS ngày 23-10 đưa tin , trong q...","Theo điều_tra ban_đầu , chiếc két sắt này nằm ...",phát_hiện két sắt chứa kg vàng dưới con đập...,Thế giới
2,6.370352e+17,"Máy bay rơi ở Mexico, ít nhất 5 người thiệt mạng","Theo Sputnik ngày 24-10 đưa tin, chiếc máy bay...",http://congan.com.vn/quoc-te/thoi-su-quoc-te/m...,congan.com.vn,102,"Máy_bay rơi ở Mexico , ít_nhất 5 người thiệt_mạng","( CAO ) Theo Sputnik ngày 24-10 đưa tin , chiế...",Chiếc máy_bay đang trong hành_trình từ bang Du...,máy_bay rơi ở mexico ít_nhất người thiệt_m...,Thế giới
3,6.372364e+17,Hình ảnh Đệ nhất phu nhân Triều Tiên xinh đẹp ...,- Giữa lúc tin đồn đang rộ lên xung quanh việc...,http://vnmedia.vn/tin-anh/201910/hinh-anh-de-n...,vnmedia.vn,102,Hình_ảnh Đệ nhất phu_nhân Triều_Tiên xinh_đẹp ...,- Giữa lúc tin_đồn đang rộ lên xung_quanh việc...,HÌnh_ảnh Đệ nhất phu_nhân xinh_đẹp của Triều_T...,hình_ảnh đệ nhất phu_nhân triều_tiên xinh_đẹp ...,Thế giới
4,6.380423e+17,Thủ lĩnh cao nhất của IS nghi bị tiêu diệt tro...,"​(CAO) Hôm 27-10, Reuters đưa tin thủ lĩnh tối...",http://congan.com.vn/quoc-te/thoi-su-quoc-te/t...,congan.com.vn,102,Thủ_lĩnh cao nhất của IS nghi bị tiêu_diệt tro...,"​ ( CAO ) Hôm 27-10 , Reuters đưa tin thủ_lĩnh...","Các nguồn_tin ở Syria , Iraq và Iran đã tiết_l...",thủ_lĩnh cao nhất của is nghi bị tiêu_diệt tro...,Thế giới
...,...,...,...,...,...,...,...,...,...,...,...
84225,6.504969e+17,CEO Apple gửi lời tri ân “phù thủy” Steve Jobs...,Nhân kỷ niệm 8 năm ngày mất nhà của nhà đồng s...,https://tinnhanhchungkhoan.vn/doanh-nhan/ceo-a...,tinnhanhchungkhoan.vn,116,NaN,NaN,NaN,ceo apple gửi lời tri_ân phù_thuỷ steve_jo...,Thế giới doanh nhân
84226,6.504969e+17,Khai mạc Diễn đàn Doanh nhân Việt Nam “Tổ quốc...,"Diễn đàn nhằm mục đích trao đổi về các cơ hội,...",https://tinnhanhchungkhoan.vn/doanh-nhan/khai-...,tinnhanhchungkhoan.vn,116,NaN,NaN,NaN,khai_mạc diễn_đàn doanh_nhân việt_nam tổ_quố...,Thế giới doanh nhân
84227,6.504969e+17,Sứ mệnh doanh nhân và những đòi hỏi của thời cuộc,"1.000 doanh nhân cả nước đến Đà Nẵng hôm nay, ...",https://tinnhanhchungkhoan.vn/doanh-nhan/su-me...,tinnhanhchungkhoan.vn,116,NaN,NaN,NaN,sứ_mệnh doanh_nhân và những đòi_hỏi của thời_c...,Thế giới doanh nhân
84228,6.504969e+17,Tỷ phú Mỹ khởi nghiệp ở tuổi nào?,"Bill Gates kinh doanh khi học trung học, Mark ...",https://tinnhanhchungkhoan.vn/doanh-nhan/ty-ph...,tinnhanhchungkhoan.vn,116,NaN,NaN,NaN,tỷ_phú mỹ khởi_nghiệp ở tuổi nào bill_gate...,Thế giới doanh nhân


In [ ]:
df.columns

Index(['id', 'title', 'sapo', 'url', 'source', 'pega_cate_id', 'title_token',
       'sapo_token', 'content_token', 'all_token', 'label'],
      dtype='object')

In [ ]:
df.to_parquet('./data/articles.parquet')

**End preprocessing articles**

# Spark read parquet

In [ ]:
parquet_df = spark.read.parquet('./data/articles.parquet')

In [ ]:
parquet_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------------------+--------------------+--------------------+--------------------+--------+
|                  id|               title|                sapo|                 url|        source|pega_cate_id|         title_token|          sapo_token|       content_token|           all_token|   label|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------------------+--------------------+--------------------+--------------------+--------+
|6.366651292038185E17|Anh phát hiện 39 ...|Ngày 23/10, cảnh ...|http://vnmedia.vn...|    vnmedia.vn|         102|Anh phát_hiện 39 ...|Ngày 23/10 , cảnh...|Theo cảnh_sát địa...|anh phát_hiện    ...|Thế giới|
|6.368640877043630...|Phát hiện két sắt...|Theo TASS ngày 23...|http://congan.com...| congan.com.vn|         102|Phát_hiện két sắt...|( CAO ) Theo TASS...|Theo điều_tra ban

In [ ]:
df_new = parquet_df.drop('id', 
                        'title', 
                        'sapo',
                        'url',
                        'source',
                        'title_token',
                        'sapo_token',
                        'content_token')

In [ ]:
df_new = df_new.dropna()

In [ ]:
df_new.show(5)

+------------+--------------------+--------+
|pega_cate_id|           all_token|   label|
+------------+--------------------+--------+
|         102|anh phát_hiện    ...|Thế giới|
|         102|phát_hiện két sắt...|Thế giới|
|         102|máy_bay rơi ở mex...|Thế giới|
|         102|hình_ảnh đệ nhất ...|Thế giới|
|         102|thủ_lĩnh cao nhất...|Thế giới|
+------------+--------------------+--------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, StringIndexer
from pyspark.ml import Pipeline

**Rename label columns**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', 
                                          metricName='accuracy')

In [ ]:
df_new = df_new.withColumnRenamed("pega_cate_id", "cate_id")
df_new = df_new.withColumnRenamed("label", "label_name")
df_new.columns

['cate_id', 'all_token', 'label_name']

In [ ]:
df_new.show(5)

+-------+--------------------+----------+
|cate_id|           all_token|label_name|
+-------+--------------------+----------+
|    102|anh phát_hiện    ...|  Thế giới|
|    102|phát_hiện két sắt...|  Thế giới|
|    102|máy_bay rơi ở mex...|  Thế giới|
|    102|hình_ảnh đệ nhất ...|  Thế giới|
|    102|thủ_lĩnh cao nhất...|  Thế giới|
+-------+--------------------+----------+
only showing top 5 rows



In [ ]:
df_new.printSchema()

root
 |-- cate_id: long (nullable = true)
 |-- all_token: string (nullable = true)
 |-- label_name: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col, column
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.pipeline import PipelineModel
def valuetoLabel(value):
    return int(value) - 101
udfValuetoLabel = udf(valuetoLabel, StringType())
df_data = df_new.withColumn('label', udfValuetoLabel('cate_id'))
df_data=df_data.withColumn('label',df_data['label'].cast("float").alias('label'))
df_data.show(5)

+-------+--------------------+----------+-----+
|cate_id|           all_token|label_name|label|
+-------+--------------------+----------+-----+
|    102|anh phát_hiện    ...|  Thế giới|  1.0|
|    102|phát_hiện két sắt...|  Thế giới|  1.0|
|    102|máy_bay rơi ở mex...|  Thế giới|  1.0|
|    102|hình_ảnh đệ nhất ...|  Thế giới|  1.0|
|    102|thủ_lĩnh cao nhất...|  Thế giới|  1.0|
+-------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:
df_data.printSchema()

root
 |-- cate_id: long (nullable = true)
 |-- all_token: string (nullable = true)
 |-- label_name: string (nullable = true)
 |-- label: float (nullable = true)



In [ ]:
df_train, df_test = df_data.randomSplit([0.8, 0.2])
df_train.show(5)

+-------+--------------------+----------+-----+
|cate_id|           all_token|label_name|label|
+-------+--------------------+----------+-----+
|    101|       đến trái_đ...|    Xã hội|  0.0|
|    101|    cây xà_cừ hàn...|    Xã hội|  0.0|
|    101|    luật_sư ở tp ...|    Xã hội|  0.0|
|    101|    người chết   ...|    Xã hội|  0.0|
|    101|    người chết vì...|    Xã hội|  0.0|
+-------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:

tkn = Tokenizer().setInputCol("all_token").setOutputCol("content_tokenized")
hashingTF = HashingTF(inputCol="content_tokenized", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
lr = LogisticRegression(regParam=0.3, elasticNetParam=0)        # Classifier 
pipeline = Pipeline(stages=[tkn, hashingTF, idf, lr])
model = pipeline.fit(df_train)
pred = model.transform(df_test)

In [ ]:
pred.show(5)

+-------+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|cate_id|           all_token|label_name|label|   content_tokenized|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-------+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    101|       năm tăng_t...|    Xã hội|  0.0|[, , , , , , , nă...|(10000,[171,187,2...|(10000,[171,187,2...|[3.33403215767116...|[0.22877952350958...|       0.0|
|    101|       đến trái_đ...|    Xã hội|  0.0|[, , , , , , , đế...|(10000,[29,127,19...|(10000,[29,127,19...|[0.89289390738834...|[0.03478179194520...|       6.0|
|    101|    cây xà_cừ hàn...|    Xã hội|  0.0|[, , , , cây, xà_...|(10000,[92,158,19...|(10000,[92,158,19...|[1.61975587440537...|[0.07031554790280...|      43.0|
|    101|    côn

In [ ]:
model.save('./trained_models/articles')

In [ ]:
predictor = PipelineModel.load('/content/drive/MyDrive/bigdata/trained_models/articles')

In [ ]:
loaded_pred = predictor.transform(df_test)

In [ ]:
loaded_pred.show(5)

+-------+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|cate_id|           all_token|label_name|label|   content_tokenized|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-------+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    101|    cây xà_cừ hàn...|    Xã hội|  0.0|[, , , , cây, xà_...|(10000,[92,158,19...|(10000,[92,158,19...|[1.60306315416585...|[0.06889387819686...|      43.0|
|    101|    luật_sư ở tp ...|    Xã hội|  0.0|[, , , , luật_sư,...|(10000,[14,255,37...|(10000,[14,255,37...|[1.43820073718851...|[0.05741624795648...|       1.0|
|    101|   công_trình ‘ x...|    Xã hội|  0.0|[, , , công_trình...|(10000,[13,14,63,...|(10000,[13,14,63,...|[3.83492433283062...|[0.02991250519426...|      14.0|
|    101|   tàu_

In [ ]:
def id2name(label_df, label_id):
    """
    Chuyển đổi label id sang label name cho toàn bộ data frame
    label_df : pandas dataframe
    label_id: list of prediction in output spark's dataframe 
    """

    pred_label_df = pd.DataFrame(label_id, columns=['Label_id']) 
    idx = label_df['id'].values
    label_name = label_df['name'].values
    label_dict = {}
    for i in range(len(idx)):
        label_dict[idx[i]] = label_name[i]
    name_list = []
    for i in pred_label_df['Label_id'].values:
        name_list.append(label_dict[i + 101])
    pred_label_df['Label_name'] = name_list
    return pred_label_df

## Test new data

In [ ]:
df_new_test = spark.read.parquet('./data/data_bigdata.parquet')

In [ ]:
df_new_test.show(5)

+------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-------+-------------------+---------------+-------+
|                id|         title_token|          sapo_token|       content_token|tag_token|        title_postag|         sapo_postag|      content_postag|tag_postag|           title_ner|            sapo_ner|         content_ner|tag_ner|        update_time|source_tracking|cate_id|
+------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-------+-------------------+---------------+-------+
|783112665224601602|Huyền My hội_ngộ ...|Show thời_trang m...|Tối_ngày 29/11 ở ...|     null|Huyền/Np My/Np hộ...|Show/Nb thời_tran...|Tối_ngày/N 29/11

In [ ]:
df_new_test.columns

['id',
 'title_token',
 'sapo_token',
 'content_token',
 'tag_token',
 'title_postag',
 'sapo_postag',
 'content_postag',
 'tag_postag',
 'title_ner',
 'sapo_ner',
 'content_ner',
 'tag_ner',
 'update_time',
 'source_tracking',
 'cate_id']

In [ ]:

df_new_test = df_new_test.drop('id',
                                'title_token',
                                'sapo_token',
                                'tag_token',
                                'title_postag',
                                'sapo_postag',
                                'content_postag',
                                'tag_postag',
                                'title_ner',
                                'sapo_ner',
                                'content_ner',
                                'tag_ner',
                                'update_time',
                                'source_tracking')

In [ ]:
df_new_test = df_new_test.withColumnRenamed('content_token', 'all_token')
df_new_test.columns

['all_token', 'cate_id']

In [ ]:
df_new_test.show(5)

+--------------------+-------+
|           all_token|cate_id|
+--------------------+-------+
|Tối_ngày 29/11 ở ...|  139.0|
|Uỷ_viên Bộ_Chính_...|  106.0|
|Với việc có cho m...|  112.0|
|Ngày 30/11 , Phòn...|  149.0|
|Messi suýt lập kỷ...|  101.0|
+--------------------+-------+
only showing top 5 rows



In [ ]:
df_new_test.count()

75396

In [ ]:
df_new_test = df_new_test.dropna()
df_new_test.count()

75367

In [ ]:
predictor = PipelineModel.load('./trained_models/articles')

In [ ]:
output_df = predictor.transform(df_new_test)

In [ ]:
output_df.select('prediction').show(5)

+----------+
|prediction|
+----------+
|      48.0|
|      42.0|
|      24.0|
|      42.0|
|      10.0|
+----------+
only showing top 5 rows



In [ ]:
label_id = output_df.select('prediction').rdd.flatMap(lambda x: x).collect()
label_df = pd.read_csv('./data/cate_info.csv')


In [ ]:
result_df = id2name(label_df, label_id)
result_df.head()

,Label_id,Label_name
0,48.0,Tin thời trang
1,42.0,Chuyện Lạ
2,24.0,Game
3,42.0,Chuyện Lạ
4,10.0,Bên lề Thể thao


+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|           all_token|cate_id|   content_tokenized|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Tối_ngày 29/11 ở ...|  139.0|[tối_ngày, 29/11,...|(10000,[199,207,2...|(10000,[199,207,2...|[0.19806684513407...|[0.01249850949017...|      48.0|
|Uỷ_viên Bộ_Chính_...|  106.0|[uỷ_viên, bộ_chín...|(10000,[12,39,47,...|(10000,[12,39,47,...|[5.64536128001971...|[2.19131816895244...|      42.0|
|Với việc có cho m...|  112.0|[với, việc, có, c...|(10000,[83,92,107...|(10000,[83,92,107...|[0.16654514333439...|[0.00210775286901...|      24.0|
|Ngày 30/11 , Phòn...|  149.0|[ngày, 30/11, ,, ...|(10000,[13,39,239...|(10000,[13,39,239...|[0.52667659872605...|[0.0

In [ ]:
!python api.py 

20/12/21 15:20:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
20/12/21 15:21:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
 * Serving Flask app "api" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:5500/ (Press CTRL+C to quit)
 * Restarting with stat
20/12/21 15:21:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLe